In [12]:
# 0. CONFIG & IMPORTS — run once at the top of your notebook/script
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from scipy import stats

sns.set(style="whitegrid", rc={"figure.figsize":(10,5)})

# Files dictionary — update paths to your CSV files (use raw strings on Windows)
files = {
    "binance": r"C:\Users\risha\Downloads\archive (9)\data_2024\binance.csv",
    "coinbase": r"C:\Users\risha\Downloads\archive (9)\data_2024\coinbase.csv",
    "kraken": r"C:\Users\risha\Downloads\archive (9)\data_2024\kraken.csv",
    "kucoin": r"C:\Users\risha\Downloads\archive (9)\data_2024\kucoin.csv"
}
# Or autodiscover from a folder:
# data_dir = "/mnt/data"
# files = {os.path.splitext(os.path.basename(p))[0]: p for p in glob.glob(os.path.join(data_dir,"*.csv"))}

OUTPUT_DIR = "crypto_analysis_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Small helper
def savefig_and_close(fig, path):
    fig.savefig(path, bbox_inches="tight", dpi=150)
    plt.close(fig)


In [13]:
# 1. PREPROCESSING
def find_first(cols, candidates):
    l = {c.lower(): c for c in cols}
    for cand in candidates:
        if cand in l:
            return l[cand]
    return None

COLUMN_CANDS = {
    "timestamp": ["timestamp", "time", "datetime", "date", "block_time", "block_timestamp"],
    "tx_id": ["tx_hash", "hash", "transaction_hash", "txid", "tx_id", "id"],
    "from_address": ["from", "from_address", "sender", "source", "src", "address_from", "sender_address"],
    "to_address": ["to", "to_address", "recipient", "destination", "dst", "address_to", "recipient_address"],
    "amount": ["value", "amount", "amt", "quantity", "token_amount", "value_usd", "amount_usd"],
    "asset": ["token", "currency", "asset", "symbol", "token_symbol"]
}

def standardize_df(df):
    df = df.copy()
    cols = list(df.columns)
    mapping = {}
    for canonical, cands in COLUMN_CANDS.items():
        found = find_first(cols, cands)
        if found:
            mapping[found] = canonical
    df = df.rename(columns=mapping)
    # Ensure required columns exist
    if "tx_id" not in df.columns:
        df["tx_id"] = df.index.astype(str)
    if "asset" not in df.columns:
        df["asset"] = "UNKNOWN"
    return df

# Load files
raw_dfs = {}
for k,p in files.items():
    try:
        df = pd.read_csv(p)
        print(f"Loaded {k}: {df.shape}")
        raw_dfs[k] = standardize_df(df)
    except Exception as e:
        print(f"Failed loading {k} from {p}: {e}")

# Quick preview
for k,df in raw_dfs.items():
    print(f"\n=== {k} columns ===\n", df.columns.tolist()[:40])
    display(df.head(2))


Loaded binance: (2000, 8)
Loaded coinbase: (2000, 8)
Loaded kraken: (2000, 8)
Loaded kucoin: (1997, 8)

=== binance columns ===
 ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'pair', 'exchange', 'tx_id', 'asset']


,timestamp,open,high,low,close,volume,pair,exchange,tx_id,asset
0,2024-10-18 15:00:00,0.03861,0.03872,0.03853,0.03861,2005.2370,ETH-BTC,Binance,0,UNKNOWN
1,2024-10-18 16:00:00,0.03860,0.03862,0.03844,0.03855,1241.2893,ETH-BTC,Binance,1,UNKNOWN



=== coinbase columns ===
 ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'pair', 'exchange', 'tx_id', 'asset']


,timestamp,open,high,low,close,volume,pair,exchange,tx_id,asset
0,2024-11-29 06:00:00,2.2789,2.2806,2.2789,2.2806,3.14,RLC-USD,Coinbase,0,UNKNOWN
1,2024-11-29 05:00:00,2.2857,2.2858,2.2721,2.2721,7485.58,RLC-USD,Coinbase,1,UNKNOWN



=== kraken columns ===
 ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'pair', 'exchange', 'tx_id', 'asset']


,timestamp,open,high,low,close,volume,pair,exchange,tx_id,asset
0,2024-10-30 07:00:00,0.243,0.243,0.243,0.243,0.243,1INCH-EUR,Kraken,0,UNKNOWN
1,2024-10-30 08:00:00,0.242,0.242,0.242,0.242,0.242,1INCH-EUR,Kraken,1,UNKNOWN



=== kucoin columns ===
 ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'pair', 'exchange', 'tx_id', 'asset']


,timestamp,open,high,low,close,volume,pair,exchange,tx_id,asset
0,2024-11-29 06:00:00,0.5888,0.5888,0.5842,0.5868,463.96,AVA-USDT,KuCoin,0,UNKNOWN
1,2024-11-29 05:00:00,0.5888,0.5888,0.5850,0.5850,86.40,AVA-USDT,KuCoin,1,UNKNOWN


In [14]:
# 2. CLEANING
def parse_ts_val(x):
    if pd.isna(x):
        return pd.NaT
    # numeric unix ts handling
    if isinstance(x,(int,float)) and x>1e9:
        s = int(x)
        if s>1e12:
            return datetime.utcfromtimestamp(s/1000.0)
        return datetime.utcfromtimestamp(s)
    try:
        return pd.to_datetime(x, infer_datetime_format=True, errors="coerce")
    except:
        return pd.NaT

cleaned_dfs = {}
for name, df in raw_dfs.items():
    c = df.copy()
    # timestamp parse
    if "timestamp" in c.columns:
        c["timestamp_parsed"] = c["timestamp"].apply(parse_ts_val)
    else:
        c["timestamp_parsed"] = pd.NaT
    # amount numeric
    if "amount" in c.columns:
        c["amount"] = pd.to_numeric(c["amount"], errors="coerce")
    else:
        c["amount"] = np.nan
    # ensure from/to exist (if not, create blank)
    if "from_address" not in c.columns:
        c["from_address"] = np.nan
    if "to_address" not in c.columns:
        c["to_address"] = np.nan
    # drop exact duplicates
    before = len(c)
    c = c.drop_duplicates(subset=["tx_id"])
    after = len(c)
    print(f"{name}: dropped {before-after} duplicate tx_ids")
    # basic derived
    c["date"] = pd.to_datetime(c["timestamp_parsed"]).dt.date
    c["hour"] = pd.to_datetime(c["timestamp_parsed"]).dt.hour
    c["log_amount"] = np.log1p(c["amount"].clip(lower=0))
    cleaned_dfs[name] = c

# Save cleaned samples
for name,c in cleaned_dfs.items():
    c.head(3).to_csv(os.path.join(OUTPUT_DIR, f"{name}_cleaned_sample.csv"), index=False)
    print("Saved sample for", name)


C:\Users\risha\AppData\Local\Temp\ipykernel_20068\2078592786.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(x, infer_datetime_format=True, errors="coerce")


binance: dropped 0 duplicate tx_ids


C:\Users\risha\AppData\Local\Temp\ipykernel_20068\2078592786.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(x, infer_datetime_format=True, errors="coerce")


coinbase: dropped 0 duplicate tx_ids


C:\Users\risha\AppData\Local\Temp\ipykernel_20068\2078592786.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(x, infer_datetime_format=True, errors="coerce")


kraken: dropped 0 duplicate tx_ids


C:\Users\risha\AppData\Local\Temp\ipykernel_20068\2078592786.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(x, infer_datetime_format=True, errors="coerce")


kucoin: dropped 0 duplicate tx_ids
Saved sample for binance
Saved sample for coinbase
Saved sample for kraken
Saved sample for kucoin


In [15]:
# 3. ANALYSIS
# Per-exchange EDA & plotting
for name, df in cleaned_dfs.items():
    print(f"\n--- ANALYSIS: {name} ---")
    print("Rows:", len(df))
    print("Valid timestamps:", df['timestamp_parsed'].notna().sum())
    print("Valid amounts:", df['amount'].notna().sum())
    print("Total volume:", df['amount'].sum(skipna=True))
    print("Unique assets:", df['asset'].nunique())

    # Amount distribution plot
    fig = plt.figure()
    sns.histplot(df["amount"].dropna(), bins=100, log_scale=(False,True))
    plt.title(f"{name} — Transaction Amount Distribution")
    savefig_and_close(fig, os.path.join(OUTPUT_DIR, f"{name}_amount_hist.png"))

    # Daily tx counts & volume
    if df['timestamp_parsed'].notna().any():
        daily = df.groupby("date").agg(daily_tx=("tx_id","count"), daily_vol=("amount","sum")).reset_index()
        fig = plt.figure()
        plt.plot(pd.to_datetime(daily['date']), daily['daily_tx'])
        plt.title(f"{name} — Daily Transaction Count")
        savefig_and_close(fig, os.path.join(OUTPUT_DIR, f"{name}_daily_tx.png"))

        fig = plt.figure()
        plt.plot(pd.to_datetime(daily['date']), daily['daily_vol'])
        plt.title(f"{name} — Daily Transaction Volume")
        savefig_and_close(fig, os.path.join(OUTPUT_DIR, f"{name}_daily_volume.png"))
        # store daily for summary
        daily.to_csv(os.path.join(OUTPUT_DIR, f"{name}_daily_summary.csv"), index=False)

    # Top senders & receivers by volume
    if df["from_address"].notna().any():
        top_senders = df.groupby("from_address").agg(tx_count=("tx_id","count"), total_out=("amount","sum")).reset_index().sort_values("total_out", ascending=False).head(20)
        top_senders.to_csv(os.path.join(OUTPUT_DIR, f"{name}_top_senders.csv"), index=False)
    if df["to_address"].notna().any():
        top_receivers = df.groupby("to_address").agg(tx_count=("tx_id","count"), total_in=("amount","sum")).reset_index().sort_values("total_in", ascending=False).head(20)
        top_receivers.to_csv(os.path.join(OUTPUT_DIR, f"{name}_top_receivers.csv"), index=False)

    # Simple anomaly detection: z-score + IsolationForest
    df_for_anom = df.dropna(subset=["amount"]).copy()
    if len(df_for_anom) >= 50:
        df_for_anom["amount_z"] = stats.zscore(df_for_anom["amount"].fillna(0))
        df_for_anom["z_outlier"] = df_for_anom["amount_z"].abs() > 4.0

        # Isolation Forest on amount & log_amount
        X = df_for_anom[["amount","log_amount"]].fillna(0).values
        scaler = StandardScaler()
        Xs = scaler.fit_transform(X)
        iso = IsolationForest(n_estimators=200, contamination=0.01, random_state=42)
        preds = iso.fit_predict(Xs)
        df_for_anom["iso_outlier"] = preds == -1

        anomalies = df_for_anom[(df_for_anom["z_outlier"]) | (df_for_anom["iso_outlier"])]
        anomalies.to_csv(os.path.join(OUTPUT_DIR, f"{name}_anomalies.csv"), index=False)
        print(f"Anomalies flagged: {len(anomalies)}. Saved to {name}_anomalies.csv")
    else:
        print("Not enough numeric rows for anomaly detection")

    # Graph analysis (if addresses exist)
    if df["from_address"].notna().any() and df["to_address"].notna().any():
        edges = df.dropna(subset=["from_address","to_address"])[["from_address","to_address","amount"]].copy()
        G = nx.DiGraph()
        for _, r in edges.iterrows():
            u,v,a = r["from_address"], r["to_address"], float(r["amount"]) if not pd.isna(r["amount"]) else 0.0
            if G.has_edge(u,v):
                G[u][v]["weight"] += a
                G[u][v]["count"] += 1
            else:
                G.add_edge(u,v, weight=a, count=1)
        print("Graph nodes:", len(G.nodes()), "edges:", len(G.edges()))
        # PageRank
        try:
            pr = nx.pagerank(G, max_iter=200)
            pr_df = pd.DataFrame.from_dict(pr, orient="index", columns=["pagerank"]).reset_index().rename(columns={"index":"address"})
            pr_df = pr_df.sort_values("pagerank", ascending=False).head(50)
            pr_df.to_csv(os.path.join(OUTPUT_DIR, f"{name}_pagerank_top50.csv"), index=False)
            print("Saved pagerank top50")
        except Exception as e:
            print("Pagerank failed:", e)



--- ANALYSIS: binance ---
Rows: 2000
Valid timestamps: 2000
Valid amounts: 0
Total volume: 0.0
Unique assets: 1
Not enough numeric rows for anomaly detection

--- ANALYSIS: coinbase ---
Rows: 2000
Valid timestamps: 2000
Valid amounts: 0
Total volume: 0.0
Unique assets: 1
Not enough numeric rows for anomaly detection

--- ANALYSIS: kraken ---
Rows: 2000
Valid timestamps: 2000
Valid amounts: 0
Total volume: 0.0
Unique assets: 1
Not enough numeric rows for anomaly detection

--- ANALYSIS: kucoin ---
Rows: 1997
Valid timestamps: 1997
Valid amounts: 0
Total volume: 0.0
Unique assets: 1
Not enough numeric rows for anomaly detection


In [16]:
# -----------------------------
# 4. INSIGHT EXTRACTION (fixed)
# -----------------------------
import math
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Expecting:
# - cleaned_dfs: dict of {exchange_name: DataFrame} from previous cleaning step
# - OUTPUT_DIR: directory path where outputs should be saved
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1) Combine cleaned data safely
combined = pd.concat(cleaned_dfs.values(), ignore_index=True, sort=False)
combined.to_csv(os.path.join(OUTPUT_DIR, "combined_cleaned.csv"), index=False)
print("Saved combined_cleaned.csv — rows:", len(combined))

# 2) Prepare address aggregation input
# Ensure address columns exist
if "from_address" not in combined.columns:
    combined["from_address"] = pd.NA
if "to_address" not in combined.columns:
    combined["to_address"] = pd.NA

# Normalize address columns to pandas string dtype to avoid dtype mismatches
combined["from_address"] = combined["from_address"].astype("string")
combined["to_address"]   = combined["to_address"].astype("string")

# Replace genuinely missing-like values with <NA> so joins behave consistently
combined["from_address"] = combined["from_address"].replace({"": pd.NA, "nan": pd.NA})
combined["to_address"]   = combined["to_address"].replace({"": pd.NA, "nan": pd.NA})

# 3) Compute outbound aggregates (per-sender)
if "tx_id" not in combined.columns:
    combined["tx_id"] = combined.index.astype(str)

out_agg = pd.DataFrame()
in_agg  = pd.DataFrame()

if combined["from_address"].notna().any():
    out_agg = (combined
               .dropna(subset=["from_address"])
               .groupby("from_address", dropna=True)
               .agg(out_tx_count = ("tx_id", "nunique"),
                    out_total    = ("amount", "sum"),
                    out_mean     = ("amount", "mean"))
               .reset_index()
               .rename(columns={"from_address": "address"}))
else:
    out_agg = pd.DataFrame(columns=["address","out_tx_count","out_total","out_mean"])

if combined["to_address"].notna().any():
    in_agg = (combined
              .dropna(subset=["to_address"])
              .groupby("to_address", dropna=True)
              .agg(in_tx_count = ("tx_id", "nunique"),
                   in_total    = ("amount", "sum"),
                   in_mean     = ("amount", "mean"))
              .reset_index()
              .rename(columns={"to_address": "address"}))
else:
    in_agg = pd.DataFrame(columns=["address","in_tx_count","in_total","in_mean"])

# Coerce 'address' columns to string in both before merging
out_agg["address"] = out_agg["address"].astype("string")
in_agg["address"]  = in_agg["address"].astype("string")

# 4) Merge inbound/outbound features
addr_feat = pd.merge(out_agg, in_agg, on="address", how="outer")
# fill numeric NaNs with 0
num_cols = [c for c in addr_feat.columns if c not in ("address",)]
addr_feat[num_cols] = addr_feat[num_cols].fillna(0)

# 5) Net flow and total tx count
addr_feat["net_flow"] = addr_feat["in_total"] - addr_feat["out_total"]
addr_feat["tx_count_total"] = addr_feat["in_tx_count"] + addr_feat["out_tx_count"]

# 6) Median interarrival (sending behavior) for addresses with timestamp
ia = []
if "timestamp_parsed" in combined.columns and combined["timestamp_parsed"].notna().any():
    # Work with rows that have both from_address and timestamp
    tmp = combined.dropna(subset=["from_address","timestamp_parsed"]).sort_values("timestamp_parsed")
    # convert timestamp_parsed to integer seconds for diff
    tmp_times = tmp.assign(ts_seconds = pd.to_datetime(tmp["timestamp_parsed"]).astype("int64") // 1_000_000_000)
    for addr, grp in tmp_times.groupby("from_address", dropna=True):
        times = grp["ts_seconds"].to_numpy()
        if len(times) > 1:
            diffs = np.diff(times)
            median_diff = float(np.median(diffs))
            ia.append((str(addr), median_diff, int(len(times))))
# Build ia_df
ia_df = pd.DataFrame(ia, columns=["address","median_interarrival_s","out_tx_count_for_ia"])
if not ia_df.empty:
    ia_df["address"] = ia_df["address"].astype("string")

# 7) Merge interarrival into addr_feat safely (coerce address dtype first)
addr_feat["address"] = addr_feat["address"].astype("string")
if not ia_df.empty:
    addr_feat = addr_feat.merge(ia_df, on="address", how="left")
else:
    # ensure columns exist
    addr_feat["median_interarrival_s"] = pd.NA
    addr_feat["out_tx_count_for_ia"] = 0

# Fill missing median_interarrival_s with a large number (so low burstiness)
addr_feat["median_interarrival_s"] = addr_feat["median_interarrival_s"].fillna(1e9).astype(float)
addr_feat["out_tx_count_for_ia"] = addr_feat["out_tx_count_for_ia"].fillna(0).astype(int)

# 8) Save address-level features
addr_feat.to_csv(os.path.join(OUTPUT_DIR, "address_level_features.csv"), index=False)
print("Saved address_level_features.csv — rows:", len(addr_feat))

# 9) Create a suspiciousness heuristic score (normalized ranks)
# Components: net_flow (higher inflow-outflow), tx_count_total (higher activity), burstiness (low median_interarrival)
# Normalize each component to percentile ranks (0..1)
def pct_rank(series):
    if series.nunique() <= 1:
        return pd.Series(0.0, index=series.index)
    return series.rank(pct=True)

# Use absolute net_flow magnitude because large in or outflows can be suspicious; here we prefer net outflow magnitude
addr_feat["_net_flow_mag"] = addr_feat["net_flow"].abs()
addr_feat["_rank_net_flow"] = pct_rank(addr_feat["_net_flow_mag"])
addr_feat["_rank_tx_count"] = pct_rank(addr_feat["tx_count_total"])

# For burstiness: smaller median_interarrival -> more bursty -> higher suspiciousness
# Convert median_interarrival_s to inverse and rank
addr_feat["_inv_interarrival"] = 1.0 / (addr_feat["median_interarrival_s"] + 1.0)   # +1 to avoid div by zero
addr_feat["_rank_burst"] = pct_rank(addr_feat["_inv_interarrival"])

# Weighted score (weights can be tuned)
w_net = 0.45
w_tx  = 0.35
w_bst = 0.20

addr_feat["suspicious_score"] = (
    addr_feat["_rank_net_flow"] * w_net +
    addr_feat["_rank_tx_count"] * w_tx +
    addr_feat["_rank_burst"] * w_bst
)

# 10) Rank and save top suspicious addresses
suspicious_addresses = addr_feat.sort_values("suspicious_score", ascending=False).reset_index(drop=True)
suspicious_addresses.to_csv(os.path.join(OUTPUT_DIR, "suspicious_addresses_ranked.csv"), index=False)
print("Saved suspicious_addresses_ranked.csv — top rows:")
display(suspicious_addresses.head(10))

# 11) Save top-n lists for manual review
suspicious_addresses.head(200).to_csv(os.path.join(OUTPUT_DIR, "suspicious_addresses_top200.csv"), index=False)
addr_feat.sort_values("tx_count_total", ascending=False).head(200).to_csv(os.path.join(OUTPUT_DIR, "most_active_addresses_top200.csv"), index=False)

# 12) High-level aggregate insights
print("\n=== AGGREGATE INSIGHTS ===")
# Top assets by volume
if "asset" in combined.columns:
    top_assets = combined.groupby("asset")["amount"].sum().sort_values(ascending=False).head(20)
    print("\nTop assets by total volume:")
    print(top_assets)

# Top senders / receivers globally
if combined["from_address"].notna().any():
    top_senders_global = combined.groupby("from_address")["amount"].sum().sort_values(ascending=False).head(10)
    print("\nTop 10 senders by volume:")
    print(top_senders_global)

if combined["to_address"].notna().any():
    top_receivers_global = combined.groupby("to_address")["amount"].sum().sort_values(ascending=False).head(10)
    print("\nTop 10 receivers by volume:")
    print(top_receivers_global)

# Most active addresses
print("\nMost active addresses (by total tx count):")
most_active = addr_feat.sort_values("tx_count_total", ascending=False)[["address","tx_count_total"]].head(10)
print(most_active.to_string(index=False))

# 13) Recommended next actions (printed)
print("\n=== RECOMMENDED NEXT ACTIONS ===")
print("1) Manually review the top 100 suspicious addresses from suspicious_addresses_top200.csv")
print("2) For each flagged address, extract transaction trails (in/out flows) and visualize subgraph to detect layering")
print("3) Cross-check flagged addresses with public tag/blacklist databases (Chainalysis, Etherscan labels, etc.)")
print("4) Tune anomaly detection: create more features (unique counterparties, % of value to exchanges, time-of-day patterns)")
print("5) If labelled fraud data becomes available, train a supervised classifier (XGBoost/LightGBM) and evaluate")
print("6) Build an interactive visualization of the top suspicious subnetworks for investigators")

# 14) Clean up temporary helper columns before final save
for col in ["_net_flow_mag","_rank_net_flow","_rank_tx_count","_inv_interarrival","_rank_burst"]:
    if col in addr_feat.columns:
        addr_feat.drop(columns=[col], inplace=True)

addr_feat.to_csv(os.path.join(OUTPUT_DIR, "address_level_features_final.csv"), index=False)
print("\nSaved address_level_features_final.csv (cleaned)")


Saved combined_cleaned.csv — rows: 7997
Saved address_level_features.csv — rows: 0
Saved suspicious_addresses_ranked.csv — top rows:


,address,out_tx_count,out_total,out_mean,in_tx_count,in_total,in_mean,net_flow,tx_count_total,median_interarrival_s,out_tx_count_for_ia,_net_flow_mag,_rank_net_flow,_rank_tx_count,_inv_interarrival,_rank_burst,suspicious_score



=== AGGREGATE INSIGHTS ===

Top assets by total volume:
asset
UNKNOWN    0.0
Name: amount, dtype: float64

Most active addresses (by total tx count):
Empty DataFrame
Columns: [address, tx_count_total]
Index: []

=== RECOMMENDED NEXT ACTIONS ===
1) Manually review the top 100 suspicious addresses from suspicious_addresses_top200.csv
2) For each flagged address, extract transaction trails (in/out flows) and visualize subgraph to detect layering
3) Cross-check flagged addresses with public tag/blacklist databases (Chainalysis, Etherscan labels, etc.)
4) Tune anomaly detection: create more features (unique counterparties, % of value to exchanges, time-of-day patterns)
5) If labelled fraud data becomes available, train a supervised classifier (XGBoost/LightGBM) and evaluate
6) Build an interactive visualization of the top suspicious subnetworks for investigators

Saved address_level_features_final.csv (cleaned)
